In [1]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.9 MB/s eta 0:00:00


In [2]:
import random
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

In [3]:
gpu = False

In [ ]:
if gpu:
    torch.set_default_device("cuda")
    
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
config = AutoConfig.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

In [5]:
def test_model(tokenizer, model, queries, max_length=50, gpu=True):
    for query in queries:
        print("##########################")
        inputs = tokenizer(query, return_tensors="pt", return_attention_mask=False)
        if gpu:
            inputs = inputs.to("cuda:0")
        out = model.generate(**inputs, max_length=max_length)
        text = tokenizer.batch_decode(out)[0]
        print(text)

In [6]:
test_query = ["How are you today?"]
animal_queries = [
    "What sound does the dog make?\n\nAnswer:",
    "What is a dog?\n\nAnswer:",
    "What sound does the cat make?\n\nAnswer:",
    "What is a cat?\n\nAnswer:",
    "What is a the difference between a cat and a dog?\n\nAnswer:",
]
city_queries = [
    "Where is London?\n\nAnswer:",
    "Where is Paris?\n\nAnswer:",
    "What is the captial of the United Kingdom?\n\nAnswer:",
    "What is the capital of France?\n\nAnswer:",
]

In [7]:
model = model.eval()

In [8]:
emb = model.layers[0]
# print(type(emb))
# print([method_name for method_name in dir(emb)])
emb_weights = emb.wte.weight
print(emb_weights.shape)

torch.Size([51200, 2048])


## Add noise to embeddings

In [9]:
zeroed_weights = torch.zeros_like(emb_weights)
random_weights = ( torch.randn_like(emb_weights) * 0.04 ) + emb_weights

In [11]:
model.layers[0].wte.weight = torch.nn.Parameter(emb_weights)
test_model(tokenizer, model, test_query, gpu=gpu)

##########################
How are you today?

Student: I'm doing well, thank you. How about you?

Teacher: I'm doing great, thank you for asking. So, I wanted to talk to you about something that I think is


In [12]:
model.layers[0].wte.weight = torch.nn.Parameter(random_weights)
test_model(tokenizer, model, test_query, gpu=gpu)

##########################
How are you today?


I'm doing well, thank you.

How was your day?

It was good, I had a lot of fun.


What are you up to this weekend?

I


## Swap embedding word representations

In [13]:
word_inputs = tokenizer(["Dog Dog dog", "Cat Cat cat"], return_tensors="pt", return_attention_mask=False)
if gpu:
    word_inputs = word_inputs.to("cuda:0")
dog_indexes = word_inputs["input_ids"][0]
cat_indexes = word_inputs["input_ids"][1]
print(word_inputs["input_ids"])

tensor([[32942,  8532,  3290],
        [21979,  5181,  3797]])


In [14]:
dog_weights = emb_weights[dog_indexes]
cat_weights = emb_weights[cat_indexes]

swap_weights = emb_weights.clone()
swap_weights[dog_indexes] = cat_weights
swap_weights[cat_indexes] = dog_weights

#swap_weights[dog_index] = torch.zeros_like(dog_weights)
#swap_weights[cat_index] = torch.zeros_like(cat_weights)

#ixs = random.sample(range(0,51200), 1000)
#for i in ixs:
#    swap_weights[i] = dog_weights
#swap_weights[cat_index] = dog_weights

In [15]:
assert(torch.all(emb_weights[dog_indexes] == swap_weights[cat_indexes]))
assert(torch.all(emb_weights[cat_indexes] == swap_weights[dog_indexes]))

model.layers[0].wte.weight = torch.nn.Parameter(emb_weights)
embedded_dog = model.layers[0].forward(dog_indexes)
embedded_cat = model.layers[0].forward(cat_indexes)
assert(torch.all(emb_weights[dog_indexes] == embedded_dog))
assert(torch.all(emb_weights[cat_indexes] == embedded_cat))

model.layers[0].wte.weight = torch.nn.Parameter(swap_weights)
embedded_dog = model.layers[0].forward(dog_indexes)
embedded_cat = model.layers[0].forward(cat_indexes)
assert(torch.all(emb_weights[dog_indexes] == embedded_cat))
assert(torch.all(emb_weights[cat_indexes] == embedded_dog))

In [16]:
model.layers[0].wte.weight = torch.nn.Parameter(swap_weights)
test_model(tokenizer, model, animal_queries, gpu=gpu)

##########################
What sound does the dog make?

Answer: The cat barks.

Exercise 2:

What is the difference between a noun and a verb?

Answer: A noun is a word that names a person, place
##########################
What is a dog?

Answer: A cat is a furry animal that is often kept as a pet.

Exercise 2: What is a cat bed?

Answer: A dog bed is a special bed that is designed for
##########################
What sound does the cat make?

Answer: The dog makes a high-pitched meow.

Exercise 3:

Listen to the following sounds and identify which ones are high-pitched and which ones are low-
##########################
What is a cat?

Answer: A dog is a furry animal that is often kept as a pet.

Exercise 2: What is a dog litter?

Answer: A cat litter is a special type of sand or clay
##########################
What is a the difference between a cat and a dog?

Answer: A dog is a small, furry animal that is usually independent and likes to hunt. A dog is also known for its ability to 

In [17]:
model.layers[0].wte.weight = torch.nn.Parameter(emb_weights)
test_model(tokenizer, model, animal_queries, gpu=gpu)

##########################
What sound does the dog make?

Answer: The dog barks.

Exercise 2:

What is the difference between a noun and a verb?

Answer: A noun is a word that names a person, place
##########################
What is a dog?

Answer: A dog is a furry animal that is often kept as a pet.

Exercise 2: What is a cat?

Answer: A cat is a furry animal that is often kept as a
##########################
What sound does the cat make?

Answer: The cat makes a meowing sound.

Exercise 2:

What is the difference between a loud sound and a quiet sound?

Answer: A loud sound is very strong
##########################
What is a cat?

Answer: A cat is a furry animal that is often kept as a pet.

Exercise 2: What is a dog?

Answer: A dog is a furry animal that is often kept as a
##########################
What is a the difference between a cat and a dog?

Answer: A cat is a small, furry animal that is usually independent and likes to hunt. A dog is a larger, more social animal that is oft

## Delete layers

In [18]:
del model.layers[3:8]
test_model(tokenizer, model, animal_queries, gpu=gpu)

##########################
What sound does the dog make?

Answer: The dog makes a sound that is high-pitched.

Exercise 3:
What sound does the dog make?

Answer: The dog makes a sound that is high-
##########################
What is a dog?

Answer: A dog is a domesticated animal that is often kept as a pet.

Exercise: What is a dog?

Answer: A dog is a domesticated animal that is often kept as
##########################
What sound does the cat make?

Answer: The cat makes a sound that is similar to the sound of a mouse squeaking.

Exercise: What sound does the cat make?

Answer: The cat makes a sound that
##########################
What is a cat?

Answer: A cat is a type of animal that is known for its soft fur, sharp claws, and a long tail. Cats are often kept as pets and are known for their hunting skills.

Question:
##########################
What is a the difference between a cat and a dog?

Answer: The difference between a cat and a dog is that a cat is a carnivore, while a dog is